# Evaluation в GigaLogger
В этом ноутбуке мы произведем оценку нашего RAG'а с помощью датасета и мощной LLM (gpt-4o)
И не только! Мы также замерим качество ответов на обычном GigaChat (без RAG), с обычным RAG, Adaptive RAG и нашу версию RAG.
У нас в боте используется измененный Adaptive RAG.
Предыдущие шаги:
1. [Генерация синтетического датасета](1_generate_dataset.ipynb)
2. [Загрузка датасета в GigaLogger](2_gigalogger_create_dataset.ipynb)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import getpass

def get_env_var(var_name):
    if var_name in os.environ:
        return os.environ[var_name]
    else:
        return getpass.getpass(f"Enter {var_name}: ")

import sys
sys.path.append("..")  # Add the parent folder to the sys.path

load_dotenv(find_dotenv())
os.environ["LANGFUSE_HOST"] = "https://gigalogger.demo.sberdevices.ru"
os.environ["LANGFUSE_PUBLIC_KEY"] = get_env_var("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = get_env_var("LANGFUSE_SECRET_KEY")

In [2]:
from langfuse import Langfuse
langfuse = Langfuse()

## Цепочка для оценки ответов

Определим промпты для оценки ответов
Мы будем оценивать по следующим критериям:
- Похожи ли ответ нашей цепочки и корректный ответ (из датасета)
- Содержит ли ответ информацию из документов, которые мы нашли с помощью RAG
- Есть ли в ответе ссылки из документов (или из стандартного раздела ссылок)

In [3]:
from langchain_core.prompts import PromptTemplate
COT_PROMPT = PromptTemplate(
    input_variables=["query", "context", "result"], template="""Ты учитель, оценивающий тест.

Тебе дан вопрос, корректный ответ и ответ студента. Тебе нужно оценить ответ студента как ПРАВИЛЬНЫЙ или НЕПРАВИЛЬНЫЙ, основываясь на корректном ответе.
Опиши пошагово своё рассуждение, чтобы убедиться, что твой вывод правильный. Избегай просто указывать правильный ответ с самого начала.

Вот базовая информация из конкретной области этого теста:
GigaChat - это большая языковая модель (LLM) от Сбера.
GigaChat API (апи) - это API для взаимодействия с GigaChat по HTTP с помощью REST запросов.
GigaChain - это SDK на Python для работы с GigaChat API. Русскоязычный форк библиотеки LangChain.
GigaGraph - это дополнение для GigaChain, который позволяет создавать мультиагентные системы, описывая их в виде графов.
Обучение GigaChat выполняется командой разработчиков. Дообучение и файнтюнинг для конечных пользователей на данный момент не доступно.
Для получения доступа к API нужно зарегистрироваться на developers.sber.ru и получить авторизационные данные.

Опирайся на эту базовую информацию, если тебе не хватает информации для проверки теста.

Пример формата:
QUESTION: здесь вопрос
TRUE ANSWER: здесь корректный ответ
STUDENT ANSWER: здесь ответ студента
EXPLANATION: пошаговое рассуждение здесь
GRADE: CORRECT или INCORRECT здесь

Тебе будем дан только один ответ студента, не несколько.
Оценивай ответ студента ТОЛЬКО на основе их фактической точности. 
Игнорируй различия в пунктуации и формулировках между ответом студента и правильным ответом.
Ответ студента может содержать больше информации, чем правильный ответ, если в нём нет противоречивых утверждений, то он корректен. Начнём!

QUESTION: "{query}"
TRUE ANSWER: "{context}"
STUDENT ANSWER: "{result}"
EXPLANATION:"""
)

ANSWERED_ON_DOCUMENTS_PROMPT = PromptTemplate(
    input_variables=["answer", "documents"], template="""Ты учитель, оценивающий тест.
Тебе будет дан ответ студента и документы, которые были даны студенту.
Избегай просто указывать правильный ответ с самого начала.
Ты должен оценить ответ студента исходя из следующих критериев:
* Ответ студента основан на документах, которые были даны студенту
* Ответ студента содержит ссылки из документов, относящихся к вопросу или ссылки из дополнительного блока ссылок

Ответ студента: "{answer}"
Документы: "{documents}"

Ты должен всегда отвечать в таком JSON формате:
{{
"thought": "Твои рассуждения по поводу оценки. Опиши пошагово своё рассуждение, чтобы убедиться, что твой вывод правильный",
"answered_on_documents": 0 или 1, где 0 — ответ не основан на документах; 1 — ответ основан на документах,
"answer_has_links": 0 или 1, где 0 - ответ не содержит релативные ссылки; 1 — ответ содержит релативные ссылки,
}}

Начнём!"""
)

In [4]:
from langchain.evaluation import CotQAEvalChain
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# Используйте мощную модель для лучшего сравнения ответов
eval_llm = ChatOpenAI(temperature=0, model="gpt-4o-2024-08-06")

answered_on_documents_chain = ANSWERED_ON_DOCUMENTS_PROMPT | eval_llm | JsonOutputParser()
cot_chain = CotQAEvalChain.from_llm(llm=eval_llm, prompt=COT_PROMPT)

async def evaluation(query, output, expected_output, documents):
    resp1 = cot_chain._prepare_output(await cot_chain.ainvoke({
        "query": query, "context": expected_output, "result": output
    }))
    thought = f"{resp1['reasoning']}"
    score = resp1['score']
    avg_score = score
    has_links = 0
    on_documents = 0
    # Добавляем оценку наличия ссылок и соответствия информации из документов, только при наличии документов
    # Если документов нет, то мы оцениваем скорее всего small-talk ответы
    # или цепочку без RAG
    if documents:
        resp2 = await answered_on_documents_chain.with_retry().ainvoke({
            "answer": output, "documents": documents
        })
        # Вес оценки со ссылками - 0.1
        has_links = resp2['answer_has_links'] / 10
        on_documents = resp2['answered_on_documents']
        avg_score += has_links + on_documents
        avg_score /= 2.1
        thought += f"\n-----\n{resp2['thought']}"
    return {
        'reasoning': thought,
        'avg_score': avg_score,
        'cot_llm': score,
        'has_links': has_links,
        'on_documents': on_documents
    }

Проверим работу цепочки оценки ответов

In [5]:
# Тут оценка неправильного ответа от LLM
await evaluation(query="Кто главный герой книги", output="Кот", expected_output="Собака", documents=[])

{'reasoning': 'EXPLANATION: Чтобы оценить ответ студента, сначала нужно определить, кто является главным героем книги согласно корректному ответу. В данном случае, правильный ответ - "Собака". Затем сравниваем это с ответом студента, который утверждает, что главным героем является "Кот". Поскольку "Кот" не совпадает с "Собака", ответ студента не соответствует правильному ответу. Таким образом, ответ студента является неверным, так как он не отражает правильную информацию о главном герое книги.\n\nGRADE: INCORRECT',
 'avg_score': 0,
 'cot_llm': 0,
 'has_links': 0,
 'on_documents': 0}

In [20]:
# Тут оценка правильного ответа от LLM
await evaluation("Кто главный герой книги", "Кот", "Котик", [])

{'reasoning': '1. Вопрос спрашивает о главном герое книги.\n2. Корректный ответ на вопрос — "Котик".\n3. Ответ студента — "Кот".\n4. Сравнивая оба ответа, можно заметить, что "Кот" и "Котик" обозначают одно и то же животное, но в разных формах. "Котик" — это уменьшительно-ласкательная форма слова "Кот".\n5. В контексте вопроса, оба ответа указывают на одно и то же существо, и нет противоречий между ответом студента и правильным ответом.\n6. Таким образом, ответ студента можно считать правильным, так как он не искажает смысл и указывает на того же главного героя.\n\nGRADE: CORRECT',
 'avg_score': 1,
 'cot_llm': 1,
 'has_links': 0,
 'on_documents': 0}

## Оценка
### Оценка ответов с обычным GigaChat

In [7]:
from langchain_community.chat_models import GigaChat
llm = GigaChat(model="GigaChat-Pro", temperature=0.00001, profanity_check=False, max_tokens=8000)

Мы оцениваем 3 ответа на один вопрос датасета, для получения средней оценки.
Из-за того, что цепочки могут иметь внутри себя компоненты с запросами к LLM
где температура не равна 0, нам нужно получить несколько раз ответы, чтобы получить среднюю оценку.

In [22]:
import asyncio
from tqdm.asyncio import tqdm

dataset = langfuse.get_dataset("rag_dataset")

async def without_rag(item, run_name, semaphore, retries=3):
    async with semaphore:
        for _ in range(retries):
            handler = item.get_langchain_handler(run_name=run_name)
            try:
                generation = (await llm.ainvoke(input=item.input, config={"callbacks": [handler]})).content
                
                resp = await evaluation(item.input, generation, item.expected_output, [])
                handler.trace.score(
                    name="avg_score",
                    value=resp['avg_score'],
                    comment=resp['reasoning']
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=resp[score_name]
                    )
            except Exception as e:
                print(e)
                handler.trace.score(
                    name="avg_score",
                    value=0,
                    comment=str(e)
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=0
                    )

tasks = []
sem = asyncio.Semaphore(5)
name = f"llm_without_rag"

for item in dataset.items:
    tasks.append(without_rag(item, name, sem))

r = await tqdm.gather(*tasks)

 93%|█████████▎| 65/70 [06:48<00:38,  7.65s/it]

100%|██████████| 70/70 [07:15<00:00,  6.22s/it]


Первый прогон сделан. Смотрим результат...
![скриншот прогона](media/llm_without_rag.png)
Результат вышел `0.15`.
Судя по всему GigaChat хорошо справляется с вопросами сам о себе, но про GigaChain отвечает слабо.
Теперь попробуем прогнать датасет с простым RAG
### Оценка ответов GigaChat + RAG(стандартный)

In [14]:
from graph import vector_store

In [15]:
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains.question_answering.stuff_prompt import CHAT_PROMPT

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | CHAT_PROMPT
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": vector_store.as_retriever(), "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [16]:
rag_chain_with_source.invoke("Как обновить GigaChain?")['answer']

'Выполните команду bash pip install -U gigachain_community'

In [17]:
import asyncio
from tqdm.asyncio import tqdm
dataset = langfuse.get_dataset("rag_dataset")

async def with_rag(item, run_name, semaphore, retries=1):
    async with semaphore:
        for _ in range(retries):
            handler = item.get_langchain_handler(run_name=run_name)
            try:
                generation = await rag_chain_with_source.ainvoke(input=item.input, config={"callbacks": [handler]})
                resp = await evaluation(item.input, generation['answer'], item.expected_output, generation['context'])
                handler.trace.score(
                    name="avg_score",
                    value=resp['avg_score'],
                    comment=resp['reasoning']
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=resp[score_name]
                    )
            except Exception as e:
                print(e)
                handler.trace.score(
                    name="avg_score",
                    value=0,
                    comment=str(e)
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=0
                    )

tasks = []
sem = asyncio.Semaphore(5)
name = f"llm_with_rag_3"

for item in tqdm(dataset.items):
    tasks.append(with_rag(item, name, sem))

r = await tqdm.gather(*tasks)

100%|██████████| 70/70 [02:41<00:00,  2.31s/it]


Смотрим результат...
![скриншот прогона](media/llm_with_rag.png)
Результат вышел `0.46`.
### Оценка ответов GigaChat + Adaptive RAG

In [11]:
from graph import graph, GraphState

In [12]:
(await graph.ainvoke(input=GraphState(question="Как обновить GigaChain?")))['generation']

'Обновление GigaChain до последней версии можно выполнить через командную строку. Просто выполните следующую команду: `bash pip install -U gigachain_community`. Это поможет вам всегда иметь актуальную версию SDK для работы с GigaChat API.'

In [13]:
import asyncio
from tqdm.asyncio import tqdm
dataset = langfuse.get_dataset("rag_dataset")

async def with_arag(item, run_name, semaphore, retries=1):
    async with semaphore:
        for _ in range(retries):
            handler = item.get_langchain_handler(run_name=run_name)
            try:
                s = GraphState(question=item.input)
                generation = await graph.ainvoke(input=s, config={"callbacks": [handler]})
                resp = await evaluation(item.input, generation['generation'], item.expected_output, generation.get("documents", []))
                handler.trace.score(
                    name="avg_score",
                    value=resp['avg_score'],
                    comment=resp['reasoning']
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=resp[score_name]
                    )
            except Exception as e:
                print(e)
                handler.trace.score(
                    name="avg_score",
                    value=0,
                    comment=str(e)
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=0
                    )

tasks = []
sem = asyncio.Semaphore(5)
name = f"llm_with_arag_3"

for item in tqdm(dataset.items):
    tasks.append(with_arag(item, name, sem))

r = await tqdm.gather(*tasks)

100%|██████████| 70/70 [04:25<00:00,  3.79s/it]


Смотрим результат...
![скриншот прогона](media/llm_with_arag.png)
Результат вышел `0.38`.
Почему?
Дело в том, что ARAG сам выбирает относиться ли вопрос к нашей векторной базе данных,
и может отказаться от ответа, не обращаясь к ней. Здесь качество зависит от качества промпта
который направляет запрос в графе.
### Оценка ответов Support Bot

In [18]:
from graph_2 import graph as graph_2
(await graph_2.ainvoke(input=GraphState(question="Как обновить GigaChain?")))['generation']

'Для обновления GigaChain выполните команду `pip install -U gigachain_community`. \n\n**Дополнительные ресурсы:**\n\n- [Документация по API](https://developers.sber.ru/docs/ru/gigachat/api/overview)\n- Репозиторий GigaChain на GitHub с исходными кодами SDK и примерами [здесь](https://github.com/ai-forever/gigachain)\n- Документация по GigaChain [здесь](https://developers.sбер.ru/docs/ру/gigachain/overview)\n- Видео "быстрый старт по работе с GigaChat API за 1 минуту" [здесь](https://www.youtube.com/watch?v=HAg-GFKl1rc&ab_channel=SaluteTech)\n- База знаний по gigachat api [здесь](https://developers.sбе.ru/help/gigachat-api)\n- Курс по LLM GigaChat [здесь](https://courses.sберенивесите.ру/llm-gigachat/)\n- Список моделей GigaChat [здесь](https://developеры.sбер.ру/docs/pу/gigachat/models)\n- Тарифы GigaChat [здесь](https://разработчики.sбет.ру/docs/рю/gigachat/api/tariffs)'

In [19]:
import asyncio
from tqdm.asyncio import tqdm
dataset = langfuse.get_dataset("rag_dataset")

async def with_arag(item, run_name, semaphore, retries=3):
    async with semaphore:
        for _ in range(retries):
            handler = item.get_langchain_handler(run_name=run_name)
            try:
                s = GraphState(question=item.input)
                generation = await graph_2.ainvoke(input=s, config={"callbacks": [handler]})
                resp = await evaluation(item.input, generation['generation'], item.expected_output, generation.get("documents", []))
                handler.trace.score(
                    name="avg_score",
                    value=resp['avg_score'],
                    comment=resp['reasoning']
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=resp[score_name]
                    )
            except Exception as e:
                print(e)
                handler.trace.score(
                    name="avg_score",
                    value=0,
                    comment=str(e)
                )
                for score_name in ['cot_llm', 'has_links', 'on_documents']:
                    handler.trace.score(
                        name=score_name,
                        value=0
                    )

tasks = []
sem = asyncio.Semaphore(5)
name = f"llm_with_support_bot_22"

for item in tqdm(dataset.items):
    tasks.append(with_arag(item, name, sem))

r = await tqdm.gather(*tasks)

100%|██████████| 70/70 [13:51<00:00, 11.88s/it]


Смотрим результат...
![скриншот прогона](media/llm_with_support_bot.png)
Результат вышел `0.79`.